In [1]:
%load_ext Cython
import numpy as np
import matplotlib.pyplot as plt


In [56]:
%%cython
import numpy as np  #import numpy inside Cython

from libc.stdlib cimport rand, srand, RAND_MAX #rand() for linear values
cimport numpy as np
cimport cython
from libc.time cimport time
from libc.math cimport exp 
srand(time(NULL))

from mc_lib.rndm import RndmWrapper

from mc_lib.rndm cimport RndmWrapper

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef np.ndarray initialstate(int linear_size, RndmWrapper rndm): 
    # initialalise lattice with 1, -1
    cdef np.ndarray[np.int_t,
                  ndim=2,
                  negative_indices=False,
                  mode='c'] lattice = \
                    np.zeros([linear_size, linear_size], dtype=int)
                    #np.random.choice([1, -1], size=(linear_size, linear_size))
    cdef int i
    cdef int j
    for i in range(0, linear_size):
        for j in range(0, linear_size):
                lattice[i,j] = 1 if rndm.uniform() > 0.5 else -1
    return lattice


@cython.boundscheck(False) 
@cython.wraparound(False)  
cdef np.ndarray mcmove(np.ndarray config,double beta,int L, RndmWrapper rndm):
    # config = lattice configuration on current step
    # beta = 1/temperature
    # L - size of lattice
    cdef:
        int i
        int j
        # Choose a random spin
        int a = int(rndm.uniform()*L)
        int b = int(rndm.uniform()*L)
        # current spin: sigma_i
        int s =  config[a, b]
        #  count neighbours: up, down, left, right
        #  nb = sum( sigma_j )
        int nb = config[(a+1)%L,b] + config[a,(b+1)%L] + \
               config[(a-1)%L,b] + config[a,(b-1)%L]
        # delta energy  = 2 * sigma_i * sum( sigma_j )
        double dE = 2*s*nb
        double ratio = exp(-dE*beta)
    if dE < 0:
        s *= -1
    elif rndm.uniform() <= ratio:
        s *= -1
    config[a, b] = s
    return config


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire functio
cdef double calcEnergy(np.ndarray config, int L):
    # config = lattice configuration on current step
    # L - size of lattice
    cdef:
        int i
        int j
        int S
        int nb 
        double energy = 0
    # Energy = sum( -sigma_i * sum( sigma_j ))
    for i in range(L):
        for j in range(L):
            S = config[i,j]
            nb = config[(i+1)%L, j] + config[i,(j+1)%L] + \
                 config[(i-1)%L, j] + config[i,(j-1)%L]
            energy += -nb*S
    return energy/4.
    
#########################################################################
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def Isingv(linear_size,
                  steps,
                  relax,
                  corr,
                  T,
                  nt):
  # T - temperature range
  # nt - number of temperature points
    
    cdef RndmWrapper rndm = RndmWrapper((time(NULL), 0))
    
    cdef:
        np.ndarray[np.int_t, ndim=2, negative_indices=False,mode='c'] config
        float Ene
        float Mag
        float iT
        float iT2
        float E1
        float M1
        float E2
        float M2
        int count = 0
        list E = []
        list M = []
        list C = []
        list X = []
        list energies = []
        list spins = []
        list Lout = []
        list Eout = []
        list Mout = []
        list Tout = []
        double n1 = 1.0/(steps*linear_size*linear_size)
        double n2 = n1 / steps
        np.ndarray[np.int_t, ndim=2, negative_indices=False,mode='c'] config_1
        int tmp = 0
        double n1out
        double n2out
    for tt in range(nt):
        # for each temperature 
        E1 = 0
        M1 = 0
        E2 = 0
        M2 = 0
        energies = []
        spins = []
        # init lattice
        config = initialstate(linear_size, rndm)
        iT=1.0/T[tt] # inversed temperature
        iT2=iT*iT # squared temperature 
        print(T[tt])
    
        for i in range(relax):         # equilibrate
            mcmove(config, iT, linear_size, rndm)

        for i in range(steps):
            if i == 0:
                n1out = 1
                n2out = 1
            else:
                n1out = 1.0/(linear_size*linear_size)
                n2out = n1out 
            config = mcmove(config, iT, linear_size, rndm) 
            tmp += 1
            if tmp == 210:
                config_1 = config
            if tmp == 211:
                print(config_1 - config)
            Ene = calcEnergy(config, linear_size)     # calculate the energy
            Mag = np.sum(config)        # calculate the magnetization
            energies.append(Ene)
            spins.append(Mag)
            E1 = E1 + Ene
            M1 = M1 + Mag
            M2 = M2 + Mag*Mag 
            E2 = E2 + Ene*Ene
            count+=1
            if  count == corr:
                count = 0
                Lout.append(config)
                #Eout.append(E1*n1out)
                #Mout.append(M1*n1out)
                Eout.append(Ene)
                Mout.append(Mag)
                Tout.append(iT)
                config_1 = config
                #print(config_1-config)
        """
        E.append(n1*E1)
        M.append(n1*M1)
        C.append((n1*E2 - n2*E1*E1)*iT2)
        X.append((n1*M2 - n2*M1*M1)*iT)
        """
        #"E, M, C, X, energies, spins
    return  Lout, Eout, Mout, Tout

In [57]:
ntt = 5
T = np.linspace(1.5, 3.5, ntt)
size = 10
#print('get')
#Lout, Eout, Mout, Tout = Isingv(linear_size = 10, steps = 100*200, relax = 1000000, corr = 200, T=T, nt=ntt)
print('Train')
corr = 100
TrainLout, TrainEout, TrainMout, TrainTout= Isingv(linear_size = size, steps = 20*corr, relax = 1000000, corr = corr, T=T, nt=ntt)

Train
1.5
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
2.0
2.5
3.0
3.5


In [41]:
print(TrainLout[45])
print(TrainTout[45])
print(TrainMout[45])

print(TrainLout[46])
print(TrainTout[46])
print(TrainMout[46])



print(TrainLout[61] - TrainLout[60])

[[ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1 -1  1  1  1]
 [ 1  1  1  1  1 -1 -1 -1  1  1]
 [ 1  1  1  1  1  1  1 -1  1  1]
 [ 1  1  1  1  1 -1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1 -1  1  1]]
0.4000000059604645
72.0
[[ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1 -1  1  1  1]
 [ 1  1  1  1  1 -1 -1 -1  1  1]
 [ 1  1  1  1  1  1  1 -1  1  1]
 [ 1  1  1  1  1 -1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1 -1  1  1]]
0.4000000059604645
86.0
[[ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1 -1  1  1  1]
 [ 1  1  1  1  1 -1 -1 -1  1  1]
 [ 1  1  1  1  1  1  1 -1  1  1]
 [ 1  1  1  1  1 -1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1 

In [8]:
np.save('10/TrainL200.npy', TrainLout)
np.save('10/TrainE200.npy', TrainEout)
np.save('10/TrainM200.npy', TrainMout)
np.save('10/TrainT200.npy', TrainTout)

np.save('10/TestL.npy', TestLout)
np.save('10/TestE.npy', TestEout)
np.save('10/TestM.npy', TestMout)
np.save('10/TestT.npy', TestTout)

NameError: name 'TestLout' is not defined

In [9]:
print('Test')
TestLout, TestEout, TestMout, TestTout= Isingv(linear_size = size, steps = 100*corr, relax = 1000000, corr = corr, T=T, nt=100)

Test
1.5
1.52020202020202
1.5404040404040404
1.5606060606060606
1.5808080808080809
1.601010101010101
1.621212121212121
1.6414141414141414
1.6616161616161615
1.6818181818181819
1.702020202020202
1.7222222222222223
1.7424242424242424
1.7626262626262625
1.7828282828282829
1.803030303030303
1.8232323232323233
1.8434343434343434
1.8636363636363638
1.8838383838383839
1.904040404040404
1.9242424242424243
1.9444444444444444
1.9646464646464648
1.9848484848484849
2.005050505050505
2.025252525252525
2.0454545454545454
2.0656565656565657
2.0858585858585856
2.106060606060606
2.1262626262626263
2.1464646464646466
2.166666666666667
2.186868686868687
2.207070707070707
2.2272727272727275
2.2474747474747474
2.2676767676767677
2.287878787878788
2.308080808080808
2.3282828282828283
2.3484848484848486
2.3686868686868685
2.388888888888889
2.409090909090909
2.4292929292929295
2.44949494949495
2.4696969696969697
2.48989898989899
2.5101010101010104
2.5303030303030303
2.5505050505050506
2.570707070707071
2.5909

In [9]:
ntt = 40
Tfirst = np.linspace(1.5, 2.06, ntt)
Tlast = np.linspace(2.46, 3.5, ntt)
T = np.concatenate((Tfirst, Tlast), axis=0)
T = np.linspace(2.07, 2.45, 100)
size = 10
#print('get')
#Lout, Eout, Mout, Tout = Isingv(linear_size = 10, steps = 100*200, relax = 1000000, corr = 200, T=T, nt=ntt)
print('Partial')
TrainLout, TrainEout, TrainMout, TrainTout= Isingv(linear_size = size, steps = 100*size*size, relax = 1000000, corr = size*size, T=T, nt=100)

Partial
2.07
2.0738383838383836
2.0776767676767673
2.0815151515151515
2.0853535353535353
2.089191919191919
2.093030303030303
2.0968686868686865
2.1007070707070707
2.1045454545454545
2.1083838383838382
2.112222222222222
2.1160606060606058
2.11989898989899
2.1237373737373737
2.1275757575757575
2.131414141414141
2.135252525252525
2.139090909090909
2.142929292929293
2.1467676767676767
2.1506060606060604
2.154444444444444
2.1582828282828284
2.162121212121212
2.165959595959596
2.1697979797979796
2.1736363636363634
2.1774747474747476
2.1813131313131313
2.185151515151515
2.188989898989899
2.1928282828282826
2.1966666666666668
2.2005050505050505
2.2043434343434343
2.208181818181818
2.212020202020202
2.215858585858586
2.2196969696969697
2.2235353535353535
2.2273737373737372
2.2312121212121214
2.235050505050505
2.238888888888889
2.2427272727272727
2.2465656565656564
2.2504040404040406
2.2542424242424244
2.258080808080808
2.261919191919192
2.2657575757575756
2.26959595959596
2.2734343434343436
2.2

In [10]:
np.save('10/TrainCritL.npy', TrainLout)
np.save('10/TrainCritE.npy', TrainEout)
np.save('10/TrainCritM.npy', TrainMout)
np.save('10/TrainCritT.npy', TrainTout)